In [7]:
import json
import pandas as pd
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim.models
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models import KeyedVectors
import gensim.downloader as api
import zipfile
import sys
import requests, io
import re 
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Необходимо создать класс, позволяющий тестировать качество работы моделей

In [8]:
def preprocess(text:str, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if re.match(r'(\d.|\d)', lemma) is None:
                if lemma not in stop_words:
                    preprocessed_text.append(lemma)
    return preprocessed_text

def read_json(path: str):
    file = open(path)
    data = json.load(file)
    return pd.DataFrame(data)
    
punctuation_marks = ['!', ',', '(', ')', ';', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()

In [9]:
class ModelResearcher:
    def __init__(self):
        self.model = None
    
    def preprocess_and_save(self, data_df: pd.DataFrame, path: str, text_field='text') -> pd.DataFrame:
        # for preprocessing dataset. Use it only in critical cases cause it's too slow on big datasets
        data_df['preprocessed_' + text_field] = data_df.apply(lambda row: preprocess(row[text_field], punctuation_marks, stop_words, morph), axis=1)
        data_df_preprocessed = data_df.copy()
        data_df_preprocessed = data_df_preprocessed.drop(columns=[text_field], axis=1)
        data_df_preprocessed.to_json(path)
        return data_df_preprocessed
    
    def train(self, data_df: pd.DataFrame, model="w2v"):            
        if model == "w2v":
            train_part = data_df['preprocessed_texts']
            self.model = gensim.models.Word2Vec(sentences=train_part, min_count=5, vector_size=50, epochs=5)
        elif model == "fast_text":
            train_part = data_df['preprocessed_texts'].tolist()
            self.model = gensim.models.FastText(vector_size=50, min_count=5)
            self.model.build_vocab(corpus_iterable=train_part)
            self.model.train(corpus_iterable=train_part, total_examples=len(train_part), epochs=5)
        return
    
    def predict_sentences_similarity(self, sentences_1: pd.Series, sentences_2: pd.Series):
        if sentences_1.size != sentences_2.size:
            return None
        else:
            if self.model is not None:
                sentences_sim = np.zeros(sentences_1.size)
                sz = sentences_1.size
                for i in range(sz): 
                    sentences_1_words = [w for w in sentences_1[i] if w in self.model.wv.index_to_key]
                    sentences_2_words = [w for w in sentences_2[i] if w in self.model.wv.index_to_key]
                    sim = self.model.wv.n_similarity(sentences_1_words, sentences_2_words)
                    sentences_sim[i] = sim
                
                return sentences_sim
            else:
                return None
            
    def calc_f1_score(self, sim_match, sim_unmatch, match_threshold):
        (TP, FP, FN, TN) = (0, 0, 0, 0)
        for i in range(sim_match.size):
            # у всех модели из данного списка должен быть sim_match[i] >= match_threshold
            if sim_match[i] >= match_threshold: 
                TP += 1
            else:
                FN += 1

        for i in range(sim_unmatch.size):
            if sim_unmatch[i] >= match_threshold: 
                FP += 1
            else:
                TN += 1

        return round(float(2*TP / (2*TP + FP + FN)), 3)

In [10]:
data_df = read_json('./preprocessed_documents.json')
match_df = read_json('./match.json')
unmatch_df = read_json('./unmatch.json')

In [11]:
match_df['preprocessed_text_rp'] = match_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
match_df['preprocessed_text_proj'] = match_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

unmatch_df['preprocessed_text_rp'] = unmatch_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
unmatch_df['preprocessed_text_proj'] = unmatch_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

In [12]:
modelResearcher = ModelResearcher()
# data_df = modelResearcher.preprocess_and_save(data_df, './preprocessed_documents.json') !!! very much time
modelResearcher.train(data_df, model="w2v")

In [14]:
match_predict = modelResearcher.predict_sentences_similarity(match_df['preprocessed_text_rp'], match_df['preprocessed_text_proj'])
unmatch_predict = modelResearcher.predict_sentences_similarity(unmatch_df['preprocessed_text_rp'], unmatch_df['preprocessed_text_proj'])
print(f'F1-score: {modelResearcher.calc_f1_score(match_predict, unmatch_predict, 0.72)}')


F1-score: 0.76


In [17]:
# match_df.insert(loc=4, column='score', value=match_predict) 
match_df

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match,id_rpd,preprocessed_text_rp,preprocessed_text_proj
0,1000,15,АЛГОРИТМЫ БЕСПИЛОТНОГО ТРАНСПОРТА,Разработка системы движения роя дронов в прост...,0.821186,Данный курс знакомит слушателей с основами упр...,Разработка системы движения роя дронов в прост...,True,NaN,"[данный, курс, знакомить, слушатель, основа, у...","[разработка, система, движение, рой, дрон, про..."
1,1005,40,АНАЛИЗ ДАННЫХ В ИСКУССТВЕННОМ ИНТЕЛЛЕКТЕ,Роботизированная система машинного творчества ...,0.774778,Освещаемые в курсе теоретические и прикладные ...,Роботизированная система машинного творчества ...,True,NaN,"[освещать, курс, теоретический, прикладной, во...","[роботизированный, система, машинный, творчест..."
2,1009,4,РАЗРАБОТКА ПРИЛОЖЕНИЙ В РАСПРЕДЕЛЕННОЙ СРЕДЕ,Разработка модулей обработки данных для библио...,0.713568,Дисциплина посвящена изучению основных принцип...,Разработка модулей обработки данных для библио...,True,NaN,"[дисциплина, посвятить, изучение, основный, пр...","[разработка, модуль, обработка, данные, библио..."
3,1011,14,Тестирование и сопровождение программного обес...,Тестирование и документирование стенда виртуал...,0.830273,"Цели дисциплины охватывают ряд направлений, св...",Тестирование и документирование стенда виртуал...,True,NaN,"[цель, дисциплина, охватывать, ряд, направлени...","[тестирование, документирование, стенд, виртуа..."
4,1016,46,Интеллектуальные системы,Разработка интеллектуальных модулей обработки ...,0.798926,Рассматриваются основные понятия теории интелл...,Разработка интеллектуальных модулей обработки ...,True,NaN,"[рассматриваться, основной, понятие, теория, и...","[разработка, интеллектуальный, модуль, обработ..."
5,105,41,Системы управления манипуляторными роботами,Разработка подводного телеуправляемого робота ...,0.617737,Дисциплина включает изложение основ теории ман...,Разработка подводного телеуправляемого робота ...,True,NaN,"[дисциплина, включать, изложение, основа, теор...","[разработка, подводный, телеуправляемый, робот..."
6,1040,26,Управление и информатика в технических системах,Программно-аппаратное конфигурирование сети ла...,0.731785,В рамках данной дисциплины рассматриваются осн...,Программно-аппаратное конфигурирование сети ла...,True,NaN,"[рамка, дать, дисциплина, рассматриваться, осн...","[программно-аппаратный, конфигурирование, сеть..."
7,1278,9,Тестирование программного обеспечения,Тестирование конфигураций на платформах: Super...,0.787697,Дисциплина посвящена изучению теоретических ос...,Тестирование конфигураций на платформах: Super...,True,NaN,"[дисциплина, посвятить, изучение, теоретически...","[тестирование, конфигурация, платформа, superm..."
8,991,30,Машинное обучение,Разработка системы детектирования и идентифика...,0.862178,Данный курс знакомит слушателей с основными ме...,Разработка системы детектирования и идентифика...,True,NaN,"[данный, курс, знакомить, слушатель, основный,...","[разработка, система, детектирование, идентифи..."
9,985,8,Основы компьютерного дизайна,"Разработка ИС ""Расписание"" / ""Деканат"".",0.646391,В курсе рассматриваются свойства зрительного в...,"Разработка ИС ""Расписание"" / ""Деканат"". Информ...",True,NaN,"[курс, рассматриваться, свойство, зрительный, ...","[разработка, иса, ``, расписание, '', /, ``, д..."


In [18]:
unmatch_df.insert(loc=4, column='score', value=unmatch_predict) 
unmatch_df

,id_rp,id_proj,name_rp,name_proj,score,text_rp,text_proj,need_match,id_rpd,preprocessed_text_rp,preprocessed_text_proj
0,2645,11,Компоненты электронной техники,Анализ динамики COVID-19 в России,0.368714,Рассмотрены основные свойства и характеристики...,Web-приложение для отображения динамики различ...,False,NaN,"[рассмотреть, основной, свойство, характеристи...","[web-приложение, отображение, динамика, различ..."
1,2649,16,Философия,Разработка системы вскрытия объектов на кадрах...,0.466567,"Философия – гуманитарная дисциплина, изучающая...",Разработка системы вскрытия объектов на кадрах...,False,NaN,"[философия, –, гуманитарный, дисциплина, изуча...","[разработка, система, вскрытие, объект, кадр, ..."
2,2646,44,Диэлектрические материалы и приборы,Разработка системы проверки доставки писем на ...,0.304073,Рассматриваются современные представления о по...,Разработка системы проверки доставки писем на ...,False,NaN,"[рассматриваться, современный, представление, ...","[разработка, система, проверка, доставка, пись..."
3,2648,46,Конфликтология,Разработка интеллектуальных модулей обработки ...,0.494805,Дисциплина «Конфликтология» знакомит с основны...,Разработка интеллектуальных модулей обработки ...,False,NaN,"[дисциплина, «, конфликтология, », знакомить, ...","[разработка, интеллектуальный, модуль, обработ..."
4,2650,21,Теоретические основы высокочастотной электроте...,Развитие кабинетов Юридических и Физических лиц,0.128042,Рассматривается распространение электромагнитн...,В рамках проекта развития кабинетов Юридически...,False,NaN,"[рассматриваться, распространение, электромагн...","[рамка, проект, развитие, кабинет, юридический..."
5,4833,2,Социология,Разработка платформы сервисов личного кабинета...,0.470993,Курс нацелен на формирование у студентов знани...,Разработка платформы сервисов личного кабинета...,False,NaN,"[курс, нацелить, формирование, студент, знание...","[разработка, платформа, сервис, личный, кабине..."
6,4833,25,Социология,Android-приложение предобработки данных 3d-рек...,0.402599,Курс нацелен на формирование у студентов знани...,Android-приложение предобработки данных 3d-рек...,False,NaN,"[курс, нацелить, формирование, студент, знание...","[android-приложение, предобработка, данные, ви..."
7,1552,29,Методы и инструменты управления рисками органи...,Разработка программного комплекса для оценки в...,0.754153,"В данной дисциплине изучаются основы, термины ...",Разработка программного комплекса для оценки в...,False,NaN,"[дать, дисциплина, изучаться, основа, термин, ...","[разработка, программный, комплекс, оценка, ва..."
8,1557,30,Бережливые технологии в менеджменте качества,Разработка системы детектирования и идентифика...,0.732821,Целью освоения дисциплины «Бережливые технолог...,Разработка системы детектирования и идентифика...,False,NaN,"[цель, освоение, дисциплина, «, бережливый, те...","[разработка, система, детектирование, идентифи..."
9,1558,31,Оценка соответствия и аудит,Определение состояния автомобиля используя видео,0.557950,Дисциплина предназначена для изучения основ оц...,Определение состояния автомобиля используя вид...,False,NaN,"[дисциплина, предназначить, изучение, основа, ...","[определение, состояние, автомобиль, использов..."


## Проделаем то же с FastText

In [ ]:
data_df = read_json('./preprocessed_documents.json')
match_df = read_json('./match.json')
unmatch_df = read_json('./unmatch.json')
match_df['preprocessed_text_rp'] = match_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
match_df['preprocessed_text_proj'] = match_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

unmatch_df['preprocessed_text_rp'] = unmatch_df.apply(lambda row: preprocess(row['text_rp'], punctuation_marks, stop_words, morph), axis=1)
unmatch_df['preprocessed_text_proj'] = unmatch_df.apply(lambda row: preprocess(row['text_proj'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
modelResearcher = ModelResearcher()
# data_df = modelResearcher.preprocess_and_save(data_df, './preprocessed_documents.json') !!! very much time
modelResearcher.train(data_df, model="fast_text")

In [ ]:
match_predict_ft = modelResearcher.predict_sentences_similarity(match_df['preprocessed_text_rp'], match_df['preprocessed_text_proj'])
unmatch_predict_ft = modelResearcher.predict_sentences_similarity(unmatch_df['preprocessed_text_rp'], unmatch_df['preprocessed_text_proj'])
print(f'F1-score: {modelResearcher.calc_f1_score(match_predict_ft, unmatch_predict_ft, 0.72)}')

In [ ]:
# match_df.insert(loc=4, column='score', value=match_predict_ft) 
match_df

In [ ]:
# unmatch_df.insert(loc=4, column='score', value=unmatch_predict_ft) 
unmatch_df